# prompt-tuning

个人认为本质是信息增强，通过统一任务范式，激活LM特定知识

In [1]:
!pip install peft

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [1]:
import torch

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq

In [2]:
dataset = Dataset.load_from_disk("datas/alpaca_data_zh")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/models/bloom-1b4")

In [4]:
def process_func(example):
    MAX_LENGTH = 512

    instruction = "\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: "
    inputs = tokenizer(instruction)
    response = tokenizer(example["output"])
    input_ids = inputs["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = inputs["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(inputs["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

In [5]:
tokenized_data = dataset.map(process_func, remove_columns=dataset.column_names)

# hard prompt

In [7]:
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/models/bloom-1b4")

In [8]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [13]:
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit

hard_prompt_text = "下面是一段人与机器人的对话"

config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text=hard_prompt_text,
    num_virtual_tokens=len(tokenizer(hard_prompt_text)["input_ids"]),
    tokenizer_name_or_path="/root/autodl-tmp/models/bloom-1b4"
)

In [14]:
model = get_peft_model(model, config)

In [15]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [16]:
model.print_trainable_parameters()

trainable params: 14,336 || all params: 1,303,126,016 || trainable%: 0.0011


In [ ]:
args = TrainingArguments(
    output_dir="caches/PEFT-hard-prompt",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=50,
    num_train_epochs=1,
)

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
)

/tmp/ipykernel_2114/3918917228.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
trainer.train()

Step,Training Loss
50,2.640900
100,2.581100
150,2.570400
200,2.544500
250,2.501400
300,2.452000
350,2.459800
400,2.457300
450,2.417800
500,2.459600


TrainOutput(global_step=840, training_loss=2.4762854530697775, metrics={'train_runtime': 828.6914, 'train_samples_per_second': 32.41, 'train_steps_per_second': 1.014, 'total_flos': 2.938062989426688e+16, 'train_loss': 2.4762854530697775, 'epoch': 1.0})

# soft prompt

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/models/bloom-1b4")

In [7]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(46145, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (l

In [8]:
from peft import PromptTuningConfig, get_peft_model, TaskType

hard_prompt_text = "下面是一段人与机器人的对话"

config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=10,
)

In [9]:
model = get_peft_model(model, config)

In [10]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [11]:
model.print_trainable_parameters()

trainable params: 20,480 || all params: 1,303,132,160 || trainable%: 0.0016


In [14]:
args = TrainingArguments(
    output_dir="trained/PEFT-hard-prompt",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=50,
    num_train_epochs=1,
)

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
)

/tmp/ipykernel_5787/3918917228.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

Step,Training Loss
50,2.811000
100,2.804100
150,2.821700
200,2.826400
250,2.804800
300,2.773500
350,2.793600
400,2.791700
450,2.752500
500,2.793900


TrainOutput(global_step=840, training_loss=2.7906064078921364, metrics={'train_runtime': 845.2417, 'train_samples_per_second': 31.776, 'train_steps_per_second': 0.994, 'total_flos': 2.938062989426688e+16, 'train_loss': 2.7906064078921364, 'epoch': 1.0})

In [6]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/models/bloom-1b4")
peft_model = PeftModel.from_pretrained(model=model, model_id="trained/PEFT-hard-prompt/checkpoint-840")

In [13]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

Human: 考试有哪些技巧？

Assistant: 下面就为大家带来英语单词，句子，以及对话，等不同类型的英语听力理解练习，旨在帮助大家更加有效的听懂英语。接下来，我们就来看看英语单词听力的相关练习吧： 1 阅读理解练习 1. 阅读下面的一篇文章，然后回答问题。The title of the book is A man. Read it on page. A．The man b．The man c．the man d．the man 2. 阅读下面的一篇短文，然后回答问题。I just got back from Paris. It is too little to tell
